In [ ]:
import os, re, json
from sqlitedict import SqliteDict

##############################################
cores = 20
project = 'mergers'
out_path = f'{project}'
db_name = f'{project}.sqlite'
##############################################

In [ ]:
def process(x):
    project = 'mergers'
    db_name = f'{project}.sqlite'
    import pyrosetta, logging
    pyrosetta.distributed.maybe_init(extra_options='-no_optH false -mute all -ignore_unrecognized_res true -load_PDB_components false')
    from fragmenstein import Victor
    Victor.work_path = f'{project}'  # db_name
    Victor.fragmenstein_throw_on_discard= True
    Victor.fragmenstein_joining_cutoff = 5 # 10
    Victor.quick_renanimation = False
    Victor.error_to_catch = Exception
    #Victor.enable_stdout(logging.ERROR)
    Victor.enable_logfile(f'{project}.log', logging.INFO)
    Victor.log_errors()
    from sqlitedict import SqliteDict
    import json, logging
    from fragmenstein.mpro import MProVictor
    print('NEW', x)
    try:
        from rdkit import Chem
        
        def loadmol(file):
            mol = Chem.MolFromMolFile(file)
            if mol.GetProp('_Name') == '':
                mol.SetProp('_Name', file.split('/')[-1].replace('.mol',''))
            return mol
        
        frags = [loadmol(file) for file in x]
        v = Victor.combine(hits=frags,
                           pdb_filename='input/template.pdb',
                           covalent_resi='81A', # a random residue is still required for the constaint ref atom.
                           covalent_resn='CYS')
        results = SqliteDict(db_name, encode=json.dumps, decode=json.loads, autocommit=True)
        results[v.long_name] = v.summarise()
        v.make_pse()
    except Exception as error:
        name = '-'.join([file.split('/')[-1].replace('.mol','') for file in x])
        error_msg = f'{error.__class__.__name__} {error}'
        results = SqliteDict(db_name, encode=json.dumps, decode=json.loads, autocommit=True)
        name = '-'.join([file.split('/')[-1].replace('.mol','') for file in x])
        results[name] = {'error': error_msg}
        Victor.journal.critical(f'*** {error_msg}, files: {x}')
    except ConnectionError:
        pass
    print('DONE', x)
    return True

In [ ]:
# get stuff started
from multiprocessing import Pool
import itertools, random, re
pool = Pool(cores, maxtasksperchild=1)

In [ ]:
# new
results = SqliteDict(db_name, encode=json.dumps, decode=json.loads, autocommit=True)
done = list(results.keys())

hits = ['newinputs/'+file for file in os.listdir('newinputs') if '.mol' in file]
to_do = [(a, b) for a, b in itertools.permutations(hits, 2) if f'{a.split("/")[-1]}-{b.split("/")[-1]}'.replace('.mol', '') not in done]
random.shuffle(to_do)
print(len(to_do))
for pair in to_do:
    pool.apply_async(process, (pair,))

-----------------


## Decompose
Not used in final calculations

In [ ]:
import os, shutil

shutil.rmtree('frags')
os.mkdir('frags')

from rdkit import Chem
from rdkit.Chem import BRICS, AllChem
from IPython.display import display
import re
from fragmenstein.mpro import MProVictor
from scipy.special import comb


hit_names = [file for file in os.listdir('input') if '.mol' in file]

#max_combinations = 20_000
minFragmentSize = 8
frags = []
letter = 'αβγδεζηθικλμνξοπρστυφχψω'
for name in hit_names:
    mol = Chem.MolFromMolFile(os.path.join('input', name))
    # protect
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum() == 0:
            atom.SetAtomicNum(117)
    deco = BRICS.BRICSDecompose(mol,
                                returnMols=True, minFragmentSize=minFragmentSize, keepNonLeafNodes=False)
    for i, dmol in enumerate(deco):
        mod = AllChem.DeleteSubstructs(dmol, Chem.MolFromSmiles('*'))
        # deprotect
        for atom in mod.GetAtoms():
            if atom.GetAtomicNum() == 117:
                atom.SetAtomicNum(0)
        mod.SetProp('_Name', f"{name.replace('.mol', '')}_{letter[i]}")
        frags.append(mod)
    shutil.copy(os.path.join('input', name), os.path.join('frags', name))

for mol in frags:
    print(mol.GetProp('_Name'))
    try:
        Chem.MolToMolFile(mol, f"frags/{mol.GetProp('_Name')}.mol")
    except Exception as error:
        print(error)
print('DONE.')